# 6장 데이터 로딩과 저장, 파일 형식

In [12]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
pd.options.display.max_colwidth = 75
pd.options.display.max_columns = 20
np.set_printoptions(precision=4, suppress=True)
plt.rcParams["font.family"] = 'Malgun Gothic'

# 1. csv 텍스트 데이터
## 1.1 csv파일 읽기
* 판다스에서 CSV 파일을 읽기: pd.read_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


    1) header(컬럼명)가 있는 파일

In [13]:
!cat /Users/minchan/Downloads/python_basic_1-main/examples/ex1.csv

df = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex1.csv")
df

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    2) header가 없는 파일 : 컬럼을 지정
        * names 인수 - 열 이름 리스트
        * header 인수 - 헤더가 없을 경우 None으로 지정

In [15]:
pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex2.csv")

names = ["a", "b", "c", "d", "message"]
pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex2.csv", names=names)

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    3) 계층적 색인 지정하기
        * index_col 인수 - 색인으로 사용할 열 번호나 이름, 계층적 색인을 지정할 경우 리스트

In [16]:
parsed = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/csv_mindex.csv",
                     index_col=["key1", "key2"])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

    4) 여러 개의 공백으로 필드를 구분한 파일 
        * sep 인수 - 필드을 구분하기 위해 사용할 정규 표현식

In [17]:
result = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex3.txt", sep="\s+")
result

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/var/folders/kf/mf4_40q57ldgrc8_67rty2dw0000gn/T/ipykernel_4203/2327394912.py:1: SyntaxWarning: invalid escape sequence '\s'
  result = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex3.txt", sep="\s+")


,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


    5) 주석을 갖고 있는 파일
        * skiprows 인수 - 파일의 시작부터 무시할 행 수 또는 무시할 행 번호가 담긴 리스트

In [18]:
pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex4.csv", skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


    6) 누락된 값이 있는 파일 : 비어 있는 문자열, NA, NULL을 NaN으로 출력 
        * na_values 인수 - NA 값으로 처리할 값들의 목록

In [22]:
result = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex5.csv")
result
# 1) 누락된 값을 확인 : isna()함수 이용
pd.isna(result)

# 2) na_values -> 컬럼별 문자열 집합을 받아서 누락된 값 처리
sentinels = {"message": ["foo", "NA"], "something": ["two"]}
result = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex5.csv", na_values=sentinels)
print(result)
result.isna()

  something  a   b     c   d message
0       one  1   2   3.0   4     NaN
1       NaN  5   6   NaN   8   world
2     three  9  10  11.0  12     NaN


,something,a,b,c,d,message
0,False,False,False,False,False,True
1,True,False,False,True,False,False
2,False,False,False,False,False,True


    7) 텍스트 파일 조금씩 
        * nrows 인수 - 파일의 첫 일부만 읽어올 때 처음 몇 줄을 읽을 것인지 지정 
        * chunksize 인수 - TextParser 객체에서 사용할 한 번에 익을 파일의 크기

In [23]:
#pd.options.display.max_rows = 10
result = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex6.csv")

result = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex6.csv", nrows=5)
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


### [예제]  ex6.csv 파일을 순회하면서 "key"열의 값을 세기

In [31]:
chunker = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex6.csv", chunksize=1000)

tot = pd.Series([], dtype='int64') # Series 객체 생성
for piece in chunker:
    tot = tot.add(piece["key"].value_counts(), fill_value=0) # 데이터프레임의 특정 칼럼 값 추출

tot = tot.sort_values(ascending=False)
tot
tot[:10]

key
E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 1.2. csv 텍스트 파일 저장하기
* 판다스에서 CSV 파일을 저장하기: df.to_csv(path, encoding = 'utf-8') 함수를 사용
    * path 인수 : 파일시스템에서의 위치, URL, 파일객체를 나타내는 문자열
    * encoding 인수 - 유니코드 인코딩 종류을 지정


In [33]:
data = pd.read_csv("/Users/minchan/Downloads/python_basic_1-main/examples/ex5.csv")
data # dataframe에 데이터 저장
data1 = data[['a','b','d']] # 인덱싱: 데이터프레임의 일부 데이터만 추출해서 데이터프레임 생성
data1
data.to_csv("/Users/minchan/Downloads/python_basic_1-main/examples/out.csv") # 데이터프레임.to_csv(저장할 파일명)

In [34]:
data1.to_csv("/Users/minchan/Downloads/python_basic_1-main/examples/out.csv")
!cat examples/out.csv

cat: examples/out.csv: No such file or directory


In [35]:
data.to_csv("/Users/minchan/Downloads/python_basic_1-main/examples/out.csv", sep="|") # 구분자 사용
data.to_csv("/Users/minchan/Downloads/python_basic_1-main/examples/out.csv", na_rep = "NULL") #누락된 값을 원하는 값으로 지정
data.to_csv("/Users/minchan/Downloads/python_basic_1-main/examples/out.csv", index=False, header=False)
data.to_csv("/Users/minchan/Downloads/python_basic_1-main/examples/out.csv", index=False, columns=["a", "b", "c"])

!cat /Users/minchan/Downloads/python_basic_1-main/examples/out.csv


a,b,c
1,2,3.0
5,6,
9,10,11.0


# 2. HTML 데이터
## 2.1 웹 스크래핑 - read_html()

* read_html() - html 문서 내 테이블 데이터 읽어 테이블 구조의 모든 데이터를 데이터프레임으로 저장해서 반환
    * https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/


#### [예제] 미국 예금 보험 공사에서 부도 은행을 보여주는 HTML 문서 읽어오기

In [52]:
import pandas as pd
#from pandas import Series, DataFrame

#tables = pd.read_html("examples/fdic_failed_bank_list.html") 
tables = pd.read_html("https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/")
len(tables)
print(type(tables))
failures = tables[0]
print(type(failures))
failures.head()

/var/folders/kf/mf4_40q57ldgrc8_67rty2dw0000gn/T/ipykernel_4203/3733964975.py:4: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html("examples/fdic_failed_bank_list.html")


ImportError: Missing optional dependency 'lxml'.  Use pip or conda to install lxml.

In [48]:
# 1) 컬럼명 수정하기
failures.columns
failures.columns = ['BankNameBank', 'CityCity', 'StateSt', 'CertCert',
       'AcquiringInstitutionAI', 'ClosingDate', 'FundFund']
failures.head(3)

NameError: name 'failures' is not defined

In [31]:
# 2) 데이터 변환 : 문자열 -> datetime
close_timestamps = pd.to_datetime(failures["ClosingDate"])
close_timestamps
#3) 년도별 데이터 개수 추출
close_timestamps.dt.year.value_counts()

ClosingDate
2010    157
2009    140
2011     92
2012     51
2008     25
2013     24
2014     18
2002     11
2017      8
2015      8
2023      5
2016      5
2020      4
2004      4
2019      4
2001      4
2007      3
2003      3
2000      2
Name: count, dtype: int64

#### [실습] : 네이버 주식 데이터 가져오기 - read_html()
* https://www.naver.com/ -> 증권 -> 국내증시 -> 시가총액
    * https://finance.naver.com/sise/sise_market_sum.naver

In [10]:
!pip install html5lib  lxml

In [15]:
import pandas as pd

# 'euc-kr' 또는 'cp949'로 인코딩 설정하여 페이지 읽기 시도
df_list = pd.read_html("https://finance.naver.com/sise/sise_market_sum.naver",encoding='cp949')
type(df_list)
# 데이터프레임 확인
for df in df_list:
    print(df)


C:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


     0      1          2        3          4       5
0  거래량   매수호가   거래대금(백만)  시가총액(억)    영업이익(억)  PER(배)
1   시가   매도호가      전일거래량  자산총계(억)    영업이익증가율  ROE(%)
2   고가  매수총잔량      외국인비율  부채총계(억)   당기순이익(억)  ROA(%)
3   저가  매도총잔량  상장주식수(천주)   매출액(억)   주당순이익(원)  PBR(배)
4  NaN    NaN        NaN   매출액증가율  보통주배당금(원)  유보율(%)
       N       종목명       현재가     전일비     등락률     액면가       시가총액      상장주식수  \
0    NaN       NaN       NaN     NaN     NaN     NaN        NaN        NaN   
1    1.0      삼성전자   84400.0  2400.0  +2.93%   100.0  5038496.0  5969783.0   
2    2.0    SK하이닉스  187200.0  1700.0  +0.92%  5000.0  1362820.0   728002.0   
3    3.0  LG에너지솔루션  395500.0     0.0   0.00%   500.0   925470.0   234000.0   
4    4.0  삼성바이오로직스  816000.0     0.0   0.00%  2500.0   580780.0    71174.0   
..   ...       ...       ...     ...     ...     ...        ...        ...   
76  49.0  HD한국조선해양  119700.0     0.0   0.00%  5000.0    84715.0    70773.0   
77  50.0      대한항공   21850.0   200.0  -0.91%  5000.0    80

In [19]:
# 데이터프레임 탐색
df = df_list[1]
data = df[df['종목명'].notnull()] # 조건색인
data
data.head()
data.to_csv("Naver_kospi.csv")

## 3.2 웹 스크래핑 - requests

* 웹 페이지 정보 가져오기 - requests 패키지 이용
  * requests.get()
  * requests.get().status_code
  * requests.get().text


### [예제] 

In [32]:
# 1. requests 모듈 설치하기
#!pip install requests
# 2. requests 모듈 임포트하기
import requests
#3. url 가져오기
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
print(resp.status_code)
resp.text

# 4. json 내용을 딕셔너리로 변환
data = resp.json() # json의 내용을 딕셔너리 형태로 변환한 객체을 반환
print(type(data))
data[0]['title']
data[0].keys() # 키 정보 출력
# 5. 딕셔너리를 DataFrame으로 생성하기
issues = pd.DataFrame(data, columns=["number", "title",
                                      "labels", "state"])
issues.head()


200
<class 'list'>


,number,title,labels,state
0,58113,ENH: function to concatenate columns with a separator,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg==', 'url': 'https://api.g...",open
1,58110,DEPR: rename `startingMonth` to `starting_month` (argument in BQuarterB...,[],open
2,58108,ENH: Series.map() that works with a dictionary of key-list pairs,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg==', 'url': 'https://api.g...",open
3,58105,Backport PR #58087 on branch 2.2.x (BLD: Build wheels using numpy 2.0rc1),"[{'id': 129350, 'node_id': 'MDU6TGFiZWwxMjkzNTA=', 'url': 'https://api....",open
4,58104,COMPAT/TST: Ensure that numpy 2 string dtype converts to object array i...,"[{'id': 127685, 'node_id': 'MDU6TGFiZWwxMjc2ODU=', 'url': 'https://api....",open


### [문제] 네이버 블로그의 키워드 관련 검색 결과 텍스트 데이터 가져오기 - requests 모듈 사용시 한계점

In [35]:
import requests
# url = https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC

params = {
    'pageNo' : 1,
    'rangeType' : 'ALL',
    'orderBy' : 'sim',
    'keyword' : '파이썬'
}

response = requests.get('https://section.blog.naver.com/Search/Post.naver?', params=params)

print(response.status_code) # HTTP GET 방식으로 URL 파라미터 값 전달
print(response.url) # status_code 는 응답코드
print(response.text) # text에는 HTML 코드

200
https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=ALL&orderBy=sim&keyword=%ED%8C%8C%EC%9D%B4%EC%8D%AC




<!DOCTYPE html>
<html lang="ko">
<head>
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <base href="/home" />
    <meta name="robots" content="noindex,nofollow"/>
    <meta name="referrer" content="always"/>
    <meta name="format-detection" content="telephone=no">
    <link rel="shortcut icon" type="image/x-icon" href="https://section.blog.naver.com/favicon.ico?3"/>
    




 

	
		<meta property="og:title" content="파이썬 : 네이버 블로그"/>

		
			
			
			<meta property="og:image" content="https://blogimgs.pstatic.net/nblog/mylog/post/og_default_image_160610.png"/>
			
		
        <meta property="og:description" content="당신의 모든 기록을 담는 공간"/>

		
	
	

<meta property="me:feed:serviceId" content="blog" />

    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />
    <title>네이버 블로그</title>

    <link rel="stylesheet" type="text/css" href="htt

## 3.3 웹 스크래핑 - beautifulsoup
* 웹 페이지 정보 추출하기 - beautifulsoup 사용법 
  * soup.select_one()
  * soup.select()
  * get_text()
* 개발자 도구 사용

### [문제] 네이버 지식iN의 검색어 관련 검색 결과 제목 가져오기

In [38]:
import requests
from bs4 import BeautifulSoup

url = 'https://kin.naver.com/search/list.naver?query=%ED%8C%8C%EC%9D%B4%EC%8D%AC'

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    # html -> beautifulsoup 객체로 변환
    soup = BeautifulSoup(html, 'html.parser')

    # 제목 한개 선택하여 텍스트 가져오기
    title = soup.select_one('#s_content > div.section > ul > li:nth-child(1) > dl > dt > a')
    print(title.get_text())

    # 제목 여러 개 선택하여 텍스트 리스트 가져오기
    titles = soup.select('#s_content > div.section > ul > li:nth-child(n) > dl > dt > a')
    for title in titles:
        print(title.get_text())
else : 
    print(response.status_code)



비주얼스튜디오코드 파이썬 질문합니다,
비주얼스튜디오코드 파이썬 질문합니다,
맥북 파이썬 idel
파이썬 모듈 미리 불러오기
파이썬과 자바
IDLE 파이썬 단축키 알려주세요
파이썬강좌 독학할때 뭐보나요??
파이썬이 미래에 도움이 될까요?
파이썬강의 듣는데가 있나요?
파이썬 라이선스 분석
파이썬독학할 때 어떻게 공부를... 


## 3.4 웹 스크래핑 - Selenium을 이용하여 사이트 자동화하기
* selenium 사용법
  1. 시스템의 OS 및 Chrome 브라우저의 버전 확인
  2. Chrome 드라이버 다운로드 및 설치
  3. 크롬 드라이버을 프로젝트 폴더에 저장 - 크롬 드라이버의 설치 경로
     * from selenium import webdriver
     * driver = webdriver.Chrome()
* 동적으로 웹 페이지 정보 추출하기
     * driver.get("웹주소") 
  

### [예제]

In [5]:
!pip3 install selenium

  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 19.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 8.2 MB/s eta 0:00:00ta 0:00:01
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)


In [7]:
from selenium import webdriver
# Chrome 브라우저 실행
driver = webdriver.Chrome()
# 웹페이지 열기
url = "https://www.google.com"
driver.get(url)

### [문제] 구글 검색창에서 "openai"로 검색한 결과를 제목과 url로 출력하는 자동 프로그램 작성


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 1. Chrome 브라우저 실행
driver = webdriver.Chrome()

# 2. Google 홈페이지 열기
url = "https://www.google.com"
driver.get(url)

# 페이지가 완전히 로드될 때까지 2초 대기
time.sleep(5)

try:
    # 검색 상자 요소 찾기
    search_box = driver.find_element(By.NAME, "q")

    # 검색어 입력
    search_query = "OpenAI"
    search_box.send_keys(search_query)

    # Enter 키를 눌러 검색 수행
    search_box.send_keys(Keys.RETURN)

    # 검색 결과가 로드될 때까지 5초 대기
    time.sleep(5)

    # 검색 결과 추출
    search_results = driver.find_elements(By.CSS_SELECTOR, "div.g")

    # 검색 결과의 제목과 URL 인쇄
    for result in search_results:
        title = result.find_element(By.CSS_SELECTOR, "h3").text
        url = result.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
        print("제목:", title)
        print("URL:", url)
        print()

except Exception as e:
    print("오류가 발생했습니다:", e)

finally:
    # 브라우저 창 닫기
    driver.quit()


제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://openai.com/

제목: OpenAI
URL: https://namu.wiki/w/OpenAI

제목: OpenAI
URL: https://en.wikipedia.org/wiki/OpenAI

제목: 오픈AI - 위키백과, 우리 모두의 백과사전
URL: https://ko.wikipedia.org/wiki/%EC%98%A4%ED%94%88AI

제목: OpenAI
URL: https://www.youtube.com/@OpenAI

제목: OpenAI
URL: https://twitter.com/OpenAI

제목: 설명
URL: https://www.google.com/search?q=OpenAI&sca_esv=31475c1a0531b981&source=hp&ei=Xm0LZpnQBb632roP0Pyk2Ao&iflsig=ANes7DEAAAAAZgt7bsUIBJHPilalMMgyZAgXLw1v16Ob&udm=&ved=0ahUKEwiZ98PsvaKFAxW-m1YBHVA-CasQ4dUDCA8&uact=5&oq=OpenAI&gs_lp=Egdnd3Mtd2l6IgZPcGVuQUlIMlAAWA9wAHgAkAEAmAEAoAEAqgEAuAEDyAEA-AEBmAIAoAIAmAMAkgcAoAcA&sclient=gws-wiz#

